# Initialize

In [ ]:
from pupil_apriltags import Detector, Detection
from collections import defaultdict
from typing import List, Dict, Tuple, Any
from glob import glob
import numpy as np
import os
import time
import sys
import cv2
from math import atan2, degrees
import platform
import socket
import matplotlib.pyplot as plt
import math
from statsFuncs import trigonometry as trig
from statsFuncs import numbers
import pandas as pd
import utils
import multiprocessing
from joblib import Parallel, delayed
from numpy import save
import pupil_functions
import decimal
import pickle
from find_blob import find_blob

%load_ext autoreload
%autoreload 2

# load data 

In [ ]:
datapath = os.getcwd() + '/data/'

if platform.system() == 'Darwin':
    EM_path = '/Users/teresa/Dropbox/Naive_Spring_Eye_Recordings_TCB/Subject_CG/003/'    
elif platform.system() == 'Linux':
    EM_path = '/home/teresa/Documents/Naive_Spring_Eye_Recordings/Subject_CG/003/'

In [ ]:
behavioral_data = pd.read_csv(datapath + 'CG_behavioral_results.csv')

# cut file just to have the 255 rows of data
behavioral_data = behavioral_data.iloc[range(255)]

# annotation_data = pd.read_csv(datapath + 'pl_msgs.csv')
timestamp_path = EM_path + 'world_timestamps.npy'

# Set up parallel computing 

In [ ]:
num_cores = multiprocessing.cpu_count()

# Extract frames 

Create frame path using OS package
Define the name of the directory to be created

In [ ]:
video_path = EM_path + 'world.mp4'
frames_path = EM_path + 'frames'

try:
    if not os.path.exists(frames_path):
        os.mkdir(frames_path)
        print("Successfully created the directory %s " % frames_path)
    else:
        print("Directory already exists %s " % frames_path)
except OSError:
    print("Creation of the directory %s failed" % frames_path)


How many frames does the video have?

In [ ]:
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

No need to do the extract frames if they have been extracted before.

In [ ]:
if not len(os.listdir(frames_path)) == total_frames:
    utils.extract_frames(video_path, frames_path, total_frames)
else:
    print('Frames previously extracted, moving forward')



In [ ]:
#------- fixing np load issue ------
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

#------- finished np load issue ------

frames_file = datapath + 'frames.npy'
tags_file = datapath + 'tag_ids.npy'

if not os.path.exists(datapath + 'frames.npy'):
    print('Detecting tags, it will take some time...')
    frames, tag_ids = utils.detect_tags(frames_path)
    print('Done! Saving results...')
    save(frames_file, frames)
    save(tags_file, tag_ids)
    with open("frames.txt", "wb") as fp:   #Pickling
        pickle.dump(frames, fp)
    with open("tagids.txt", "wb") as fp:   #Pickling
        pickle.dump(tag_ids, fp)
    print('Done!') 
else:
    print('Tags per frame were already extracted, loading the files....')
    frames = np.load(frames_file)
    tag_ids = np.load(tags_file)
#     with open("frames.txt", "rb") as fp:   # Unpickling
#         frames_copy = pickle.load(fp)
    with open("tagids.txt", "rb") as fp:   # Unpickling
        tagids_copy = pickle.load(fp)
    print('Frames and tag IDs were loaded')

#------restore np load ------
# restore np.load for future normal usage
np.load = np_load_old

In [ ]:
# Descriptive print statements
# tag_count = sum(count for count in tagids_copy.values())
# print(f'Detected {tag_count} tags in {len(frames_copy)} frames.')
# print(f'Found IDs of {list(tagids_copy.keys())}.')

In [ ]:
correct_tags = [0, 1, 2, 3, 5, 6, 7, 8, 9, 11]

Check in how many of the frames it didn't find 10 tags

# Annotations 

In [ ]:
annotation_data = pupil_functions.load_annotations(EM_path)

In [ ]:
annotation_data[["trial", "block"]] = annotation_data[["trial", "block"]].apply(pd.to_numeric)

# Timestamps 

In [ ]:
world_timestamps = np.load(timestamp_path)
world_timestamps_df = pd.DataFrame(index=range(total_frames), columns=['FRAME','TIMESTAMP'])
world_timestamps_df['TIMESTAMP'] = world_timestamps
world_timestamps_df['FRAME'] = range(total_frames)

In [ ]:
world_timestamps_df['TIMESTAMP_ROUND'] = round(world_timestamps_df['TIMESTAMP'], 4)
annotation_data['timestamp_round'] = round(annotation_data['timestamp'], 4)

Note that first frame is frame 0

In [ ]:
# # Find timestamp where experiment begins:
# exp_beging_ts = np.float64(annotation_data.loc[annotation_data['exp_event'] == 'Block 1 Begins', 'timestamp_round'])

# # find frame for that timestamp
# exp_beging_fr = world_timestamps_df.loc[world_timestamps_df['TIMESTAMP_ROUND'] == numbers.closest(world_timestamps_df['TIMESTAMP_ROUND'], exp_beging_ts), 'FRAME']

In [ ]:
for annotation_idx in range(len(annotation_data)):
    
    msg = annotation_data.loc[annotation_idx, 'exp_event']
    msg_ts = annotation_data.loc[annotation_idx, 'timestamp']
    
    closest_ts = numbers.closest(world_timestamps_df['TIMESTAMP'], msg_ts)
    # find frame:
    msg_frame = int(world_timestamps_df.loc[world_timestamps_df['TIMESTAMP'] == closest_ts, 'FRAME'])
    
    annotation_data.loc[annotation_idx, 'frame'] = msg_frame

## Trial 1 

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4154) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4155) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4156) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4157) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4158) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4159) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4160) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4161) + '.png')
plt.imshow(image)

## Trial 2 

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4212) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4213) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4214) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4215) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4216) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4217) + '.png')
plt.imshow(image)

## Trial 3 

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4261) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4262) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4263) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4264) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4265) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4266) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4267) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4268) + '.png')
plt.imshow(image)

## Trial 4 

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4314) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4315) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4316) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4317) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4318) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4319) + '.png')
plt.imshow(image)

In [ ]:
image = cv2.imread(frames_path + '/frame' + str(4320) + '.png')
plt.imshow(image)

# Using functions

In [ ]:
fr_temp = frames_path + '/frame' + str(4319) + '.png'
center_blob, top_left_blob, bottom_right_blob = find_blob(fr_temp,correct_tags, angle=340, draw=True)

# Trial loop 

In [ ]:
trial_unique = behavioral_data.trialNumber.unique()
block_unique = behavioral_data.blockNumber.unique()

numBlocks = len(block_unique)
numTrials = len(trial_unique)

totalNumTrials = len(behavioral_data)

## Without lag correction 

In [ ]:
coordinates_df = pd.DataFrame()
utils.print_progress_bar(0, totalNumTrials, prefix='Progress:', suffix='Complete', length=50)
tot_trial = 0
row = 0
for block in block_unique:
    if not np.isnan(block):
#         print('Processing block ' + str(block) + ' out of ' + str(len(block_unique)))

        for trial in trial_unique:
            if not np.isnan(trial):
                # find frames where trial happens
                trial_data = annotation_data[(annotation_data['block'] == block) & (annotation_data['trial'] == trial)]
                image_onset_frame = int(trial_data.loc[trial_data['task'] == 'Image onset', 'frame'])

                # find where image onset finishes
                image_offset_frame = int(trial_data.loc[trial_data['task'] == 'Adjust task', 'frame'])

                # now that we have the frames, lay them in a variable:
                selected_frames = np.arange(image_onset_frame,image_offset_frame,1)

                # find angle in behavioral data
                angle_trial = int(behavioral_data.loc[
                    (behavioral_data['blockNumber'] == block) & (behavioral_data['trialNumber'] == trial), 'stimLocationDeg'])

                # we want to find the blob for all frames during image onset
                numFrames = len(selected_frames)

                count_in_trial = 0
                for frame in selected_frames:

                    # first define path to that frame
                    fr_temp = frames_path + '/frame' + str(frame) + '.png'

                    # find blob for that frame, input the angle in that trial
                    center_blob, top_left_blob, bottom_right_blob = find_blob(fr_temp,correct_tags,
                                                                              angle=angle_trial, draw=True,
                                                                             progress_bar=False)

                    # if we couldn't find at least 10 tags:
                    if center_blob == 'Error':
                        
                        # if it's not the first one in trial, just copy the one before
                        if count_in_trial != 0:
                            coordinates_df.loc[row, 'BLOCK'] = coordinates_df.loc[row-1, 'BLOCK']
                            coordinates_df.loc[row, 'TRIAL'] = coordinates_df.loc[row-1, 'TRIAL']
                            coordinates_df.loc[row, 'ANGLE'] = coordinates_df.loc[row-1, 'ANGLE']
                            coordinates_df.loc[row, 'ORDER'] = count_in_trial
                            coordinates_df.loc[row, 'CENTER_BLOB_X'] = coordinates_df.loc[row-1, 'CENTER_BLOB_X']
                            coordinates_df.loc[row, 'CENTER_BLOB_Y'] = coordinates_df.loc[row-1, 'CENTER_BLOB_Y']
                            coordinates_df.loc[row, 'TOP_LEFT_BLOB_X'] = coordinates_df.loc[row-1, 'TOP_LEFT_BLOB_X']
                            coordinates_df.loc[row, 'TOP_LEFT_BLOB_Y'] = coordinates_df.loc[row-1, 'TOP_LEFT_BLOB_Y']
                            coordinates_df.loc[row, 'BOTTOM_RIGHT_BLOB_X'] =  coordinates_df.loc[row-1, 'BOTTOM_RIGHT_BLOB_X']
                            coordinates_df.loc[row, 'BOTTOM_RIGHT_BLOB_Y'] =  coordinates_df.loc[row-1, 'BOTTOM_RIGHT_BLOB_Y']
                        
                        elif count_in_trial == 0: # if it is indeed the first trial, then have the same as next trial                   
                            coordinates_df.loc[row, 'BLOCK'] = float('nan')
                            coordinates_df.loc[row, 'TRIAL'] = float('nan')
                            coordinates_df.loc[row, 'ANGLE'] = float('nan')
                            coordinates_df.loc[row, 'ORDER'] = float('nan')
                            coordinates_df.loc[row, 'CENTER_BLOB_X'] = float('nan')
                            coordinates_df.loc[row, 'CENTER_BLOB_Y'] = float('nan')
                            coordinates_df.loc[row, 'TOP_LEFT_BLOB_X'] = float('nan')
                            coordinates_df.loc[row, 'TOP_LEFT_BLOB_Y'] = float('nan')
                            coordinates_df.loc[row, 'BOTTOM_RIGHT_BLOB_X'] =  float('nan')
                            coordinates_df.loc[row, 'BOTTOM_RIGHT_BLOB_Y'] =  float('nan')
                    else:
                        coordinates_df.loc[row, 'BLOCK'] = block
                        coordinates_df.loc[row, 'TRIAL'] = trial
                        coordinates_df.loc[row, 'ANGLE'] = angle_trial
                        coordinates_df.loc[row, 'ORDER'] = count_in_trial
                        coordinates_df.loc[row, 'CENTER_BLOB_X'] = center_blob[0]
                        coordinates_df.loc[row, 'CENTER_BLOB_Y'] = center_blob[1]
                        coordinates_df.loc[row, 'TOP_LEFT_BLOB_X'] = top_left_blob[0]
                        coordinates_df.loc[row, 'TOP_LEFT_BLOB_Y'] = top_left_blob[1]
                        coordinates_df.loc[row, 'BOTTOM_RIGHT_BLOB_X'] =  bottom_right_blob[0]
                        coordinates_df.loc[row, 'BOTTOM_RIGHT_BLOB_Y'] =  bottom_right_blob[1]


                    count_in_trial += 1
                    row += 1
                    
            tot_trial += 1
            utils.print_progress_bar(tot_trial, totalNumTrials, prefix='Progress:', suffix='Complete', length=50)




